# Iteration 9: patient count (KPI)

## Plain English Summary

The model is already tracking the number of patients. In the paper the authors compare the number of patients generated with real data.  We need to reset the patient count when the warm-up period is completed. We do this by introducing a new event that happens only once. This is called "warm-up complete".

## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [2]:
display(Markdown("inputs/iteration_9.md"))

modify the code as follows: 

Create a new generator function in the CCUModel class called "warmup_complete". It should be scheduled by simpy to run only once at the end of the warm up period.  The logic of the function should set the patient count to zero.  If trace is True the model should also print out "warm-up complete".

At the end of the simulation run add the patient count to the pandas dataframe.

Show all of the model code: this includes all patient generator functions in the CCUModel class and all code in the Experiment class.

## Date and time of input to LLM

> 1815 4th December 2023 

## Observations on the LLM Output

* The LLM has added the following method to `CCUModel`

```python
    def warmup_complete(self):
        yield self.env.timeout(self.experiment.warm_up_period)
        self.patient_count = 0
        if self.experiment.trace:
            print("Warm-up complete")
```

* The `run` method correctly schedules this event.

## LLM Output 1:

In this modified code, the CCUModel class now includes a new generator function "warmup_complete" that is scheduled to run only once at the end of the warm-up period. The logic of the function sets the patient count to zero, and if trace is True, the model prints out "warm-up complete". At the end of the simulation run, the patient count is added to the pandas dataframe. If the "trace" parameter is set to True, the model will print out simulated events such as admissions, discharges, or cancellations. The model will always print out the performance at the end of the simulated run.

In [3]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24,
                 trace=False):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.trace = trace
        self.patient_count = 0

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                if self.experiment.trace:
                    print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.experiment.cancelled_elective_count += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        arrival_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            wait_time = self.env.now - arrival_time
            if self.experiment.trace:
                print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            if self.experiment.trace:
                print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
                self.experiment.mean_waiting_time_unplanned += wait_time
                self.experiment.total_unplanned_admissions += 1
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Critical care bed is available for next patient at time {self.env.now}")

    def warmup_complete(self):
        yield self.env.timeout(self.experiment.warm_up_period)
        self.patient_count = 0
        if self.experiment.trace:
            print("Warm-up complete")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.process(self.warmup_complete())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            mean_waiting_time_unplanned = self.experiment.mean_waiting_time_unplanned / self.experiment.total_unplanned_admissions
            bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
            bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds
            performance_measures = pd.DataFrame({'Cancelled Elective Operations': [self.experiment.cancelled_elective_count], 'Bed Utilization': [bed_utilization], 'Mean Waiting Time Unplanned': [mean_waiting_time_unplanned], 'Bed Occupancy': [bed_occupancy], 'Patient Count': [self.patient_count]})
            print(performance_measures)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

   Cancelled Elective Operations  Bed Utilization  \
0                            338         0.859149   

   Mean Waiting Time Unplanned  Bed Occupancy  Patient Count  
0                    97.653851      20.619581           1600  


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [4]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [5]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [6]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [7]:
extreme_value_test_1()

   Cancelled Elective Operations  Bed Utilization  \
0                              0          0.23412   

   Mean Waiting Time Unplanned  Bed Occupancy  Patient Count  
0                          0.0       5.618873            396  


### Test 2: Extreme value test 2

PASS

In [8]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means, trace=True)
    run_test(experiment)

In [9]:
try:
    extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective surgery patient 1 arrived at time 17.977706750175514
Elective surgery patient 1 admitted to critical care bed at time 17.977706750175514
Elective surgery patient 2 arrived at time 34.41823924521725
Elective surgery patient 2 admitted to critical care bed at time 34.41823924521725
Elective surgery patient 3 arrived at time 48.70631857127839
Elective surgery patient 3 admitted to critical care bed at time 48.70631857127839
Elective surgery patient 4 arrived at time 66.88103084457728
Elective surgery patient 4 admitted to critical care bed at time 66.88103084457728
Elective surgery patient 5 arrived at time 81.01125423248494
Elective surgery patient 5 admitted to critical care bed at time 81.01125423248494
Elective surgery patient 5 discharged from critical care bed at time 82.01556666073715
Elective surgery patient 5 discharged from critical care bed at time 83.90032270335203
Critical care bed is available for next patient at time 87.01556666073715
Critical care bed is available

### Test 3: Different run length

PASS

In [10]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [11]:
test_results_collection_period(10)

   Cancelled Elective Operations  Bed Utilization  \
0                              0         0.515565   

   Mean Waiting Time Unplanned  Bed Occupancy  Patient Count  
0                          0.0       12.37355              0  


730

### Test 4: Extreme value test 3

PASS.

In [12]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)
    print(f'{experiment.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')

In [13]:
extreme_value_test_3()

   Cancelled Elective Operations  Bed Utilization  \
0                            487         0.937126   

   Mean Waiting Time Unplanned  Bed Occupancy  Patient Count  
0                  4525.069876       0.937126           1609  
experiment.total_treatment_time=8096.764634908616
experiment.results_collection_period=8640


### Test 5: Extreme value test 4

PASS

In [14]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M, trace=True)
    run_test(experiment)

In [15]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Patient 1 arrived from the Wards at time 1.3937878638996966
Patient 1 admitted to critical care bed at time 1.3937878638996966
Patient 2 arrived from the Wards at time 7.1762640558325375
Patient 2 admitted to critical care bed at time 7.1762640558325375
Patient 3 arrived from Emergency surgery at time 11.982931810510376
Patient 3 admitted to critical care bed at time 11.982931810510376
Elective surgery patient 4 arrived at time 14.866121617727801
Elective surgery patient 4 admitted to critical care bed at time 14.866121617727801
Patient 5 arrived from Accident and Emergency at time 16.246392023440748
Patient 5 admitted to critical care bed at time 16.246392023440748
Patient 6 arrived from other hospitals at time 29.700299662642806
Patient 6 admitted to critical care bed at time 29.700299662642806
Patient 7 arrived from Emergency surgery at time 36.6852080135597
Patient 7 admitted to critical care bed at time 36.6852080135597
Elective surgery patient 8 arrived at time 37.0815766636431
E

### Test 6: Vary Warm-up parameter

PASS

In [16]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [17]:
test_warmup_period(0)

   Cancelled Elective Operations  Bed Utilization  \
0                            212         0.849167   

   Mean Waiting Time Unplanned  Bed Occupancy  Patient Count  
0                    14.386403      20.380016           1610  


(8640, 8640)